In [1]:
%load_ext tensorboard

In [2]:
rm -rf ./logs/

In [3]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
from tensorflow import keras

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [5]:
ratings = pd.read_pickle("tiny_selection_longform.pkl")

In [6]:
user_enc = LabelEncoder()
ratings['user'] = user_enc.fit_transform(ratings['user_id'].values)
n_users = ratings['user'].nunique()
n_users

2390

In [7]:
item_enc = LabelEncoder()
ratings['movie'] = item_enc.fit_transform(ratings['movie_id'].values)
n_movies = ratings['movie'].nunique()
n_movies

1648

In [8]:
X_train, X_test = train_test_split(ratings[['movie', 'user', 'rating_score']], test_size=0.2, random_state=42)

In [9]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([64, 128]))
HP_DROPOUT = hp.HParam('droupout', hp.RealInterval(.6, .7))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.Discrete([.001, .01]))
# HP_ACTIVATION = hp.HParam('activation', hp.Discrete(['relu', 'sigmoid']))


METRIC = 'mean_squared_error'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_LEARNING_RATE],
        metrics=[hp.Metric(METRIC, display_name = 'mse')],
    )

In [10]:
def train_test_model(hparams, nmovie_id, nuser_id):
    
    input_movies = keras.layers.Input(shape=[1])
    embed_movies = keras.layers.Embedding(nmovie_id+1, 15)(input_movies)
    movies_out = keras.layers.Flatten()(embed_movies)

    input_users = keras.layers.Input(shape=[1])
    embed_users = keras.layers.Embedding(nuser_id+1, 15)(input_users)
    users_out = keras.layers.Flatten()(embed_users)

    conc_layer = keras.layers.Concatenate()([movies_out, users_out])
    x = keras.layers.Dense(hparams[HP_NUM_UNITS], activation='relu')(conc_layer)
    dropout = x = keras.layers.Dropout(rate=hparams[HP_DROPOUT], seed=42)(x)
    x_out = x = keras.layers.Dense(1, activation='relu')(x)

    model = keras.Model([input_movies, input_users], x_out)
    
    opt = tf.optimizers.Adam(learning_rate=hparams[HP_LEARNING_RATE])
    model.compile(optimizer=opt, loss='mean_squared_error', metrics = 'mean_squared_error')
    
    hist = model.fit([X_train.movie, X_train.user], 
                 X_train.rating_score,  
                 batch_size=1000, epochs=10, validation_split=.2)

#     print(hist.history['val_mean_squared_error'])
    return hist.history['val_mean_squared_error'][-1]

In [11]:
def run(run_dir, hparams, nmovie_id, nuser_id):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)
        mse = train_test_model(hparams, nmovie_id, nuser_id)
        tf.summary.scalar(METRIC, mse, step=1)

In [12]:
nmovie_id = ratings.movie_id.nunique()
nuser_id = ratings.user_id.nunique()

In [13]:
session_num = 0

for num_units in HP_NUM_UNITS.domain.values:
    for dropout_rate in (HP_DROPOUT.domain.min_value, HP_DROPOUT.domain.max_value):
        for learning_rate in HP_LEARNING_RATE.domain.values:
#             for activation in HP_ACTIVATION.domain.values:
            hparams = {
                HP_NUM_UNITS: num_units,
                HP_DROPOUT: dropout_rate,
                HP_LEARNING_RATE: learning_rate,
#                     HP_ACTIVATION: activation,
            }

            run_name = "run-{}".format(session_num)
            print("----Starting trial: {}".format(run_name))
            print({h.name: hparams[h] for h in hparams})
            run('logs/hparam_tuning/' + run_name, hparams, nmovie_id, nuser_id)
            session_num += 1

----Starting trial: run-0
{'num_units': 64, 'droupout': 0.6, 'learning_rate': 0.001}
Epoch 1/10
2/2 [==============================] - 3s 969ms/step - loss: 10.7858 - mean_squared_error: 10.7858 - val_loss: 10.5408 - val_mean_squared_error: 10.5408
Epoch 2/10
2/2 [==============================] - 0s 140ms/step - loss: 10.5835 - mean_squared_error: 10.5835 - val_loss: 10.4510 - val_mean_squared_error: 10.4510
Epoch 3/10
2/2 [==============================] - 0s 291ms/step - loss: 10.5356 - mean_squared_error: 10.5356 - val_loss: 10.3579 - val_mean_squared_error: 10.3579
Epoch 4/10
2/2 [==============================] - 0s 252ms/step - loss: 10.5076 - mean_squared_error: 10.5076 - val_loss: 10.2631 - val_mean_squared_error: 10.2631
Epoch 5/10
2/2 [==============================] - 0s 264ms/step - loss: 10.3232 - mean_squared_error: 10.3232 - val_loss: 10.1665 - val_mean_squared_error: 10.1665
Epoch 6/10
2/2 [==============================] - 0s 236ms/step - loss: 10.2028 - mean_squared_

Epoch 9/10
2/2 [==============================] - 0s 248ms/step - loss: 10.0322 - mean_squared_error: 10.0322 - val_loss: 9.8551 - val_mean_squared_error: 9.8551
Epoch 10/10
2/2 [==============================] - 0s 181ms/step - loss: 9.9446 - mean_squared_error: 9.9446 - val_loss: 9.7407 - val_mean_squared_error: 9.7407
----Starting trial: run-5
{'num_units': 128, 'droupout': 0.6, 'learning_rate': 0.01}
Epoch 1/10
2/2 [==============================] - 1s 395ms/step - loss: 10.6820 - mean_squared_error: 10.6820 - val_loss: 9.8387 - val_mean_squared_error: 9.8387
Epoch 2/10
2/2 [==============================] - 0s 269ms/step - loss: 9.8388 - mean_squared_error: 9.8388 - val_loss: 8.7786 - val_mean_squared_error: 8.7786
Epoch 3/10
2/2 [==============================] - 0s 226ms/step - loss: 8.4386 - mean_squared_error: 8.4386 - val_loss: 7.4360 - val_mean_squared_error: 7.4360
Epoch 4/10
2/2 [==============================] - 0s 212ms/step - loss: 6.6340 - mean_squared_error: 6.6340 - 

In [27]:
%tensorboard --logdir logs/hparam_tuning

ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.

In [28]:
import tensorboard

In [29]:
tensorboard.__file__

'/home/sahar/anaconda3/envs/tf_env/lib/python3.8/site-packages/tensorboard/__init__.py'

In [30]:
import sys

In [31]:
sys.path

['/home/sahar/Documents/Springboard-Capstone-Three',
 '/home/sahar/anaconda3/envs/tf_env/lib/python38.zip',
 '/home/sahar/anaconda3/envs/tf_env/lib/python3.8',
 '/home/sahar/anaconda3/envs/tf_env/lib/python3.8/lib-dynload',
 '',
 '/home/sahar/anaconda3/envs/tf_env/lib/python3.8/site-packages',
 '/home/sahar/anaconda3/envs/tf_env/lib/python3.8/site-packages/IPython/extensions',
 '/home/sahar/.ipython']

In [32]:
from tensorboard import notebook

In [33]:
notebook.list()

No known TensorBoard instances running.
